### Read-Write Controller with Monitors

Consider the monitor `RWController` from the course notes:
```algorithm
monitor RWcontroller
    var nr, nw = 0, 0
    {(nr = 0 ∨ nw = 0) ∧ nw ≤ 1}
    var readOK: condition  {signalling condition:  nw = 0}
    var writeOK: condition  {signalling condition:  nr = 0 ∧ nw = 0}
    procedure startRead
        while nw > 0 do wait(readOK)
        nr := nr + 1
    procedure endRead
        {nr > 0} nr := nr – 1
        if nr = 0 then signal(writeOK)
    procedure startWrite
        while nr > 0 or nw > 0 do wait(writeOK)
        nw := nw + 1
    procedure endWrite
        {nw > 0} nw := nw – 1
        signal(writeOK) ; signalAll(readOK)
```
Add to the bodies of the procedures of RWcontroller the complete annotation needed to show correctness. Argue for the correctness!

YOUR SOLUTION HERE

_Solution_:
- For `startRead`:
```algorithm
        {CI}
        while nw > 0 do
            wait(readOK) {CI}
        {CI ∧ nw = 0}
        nr := nr + 1
        {CI}
```
> With the signal-and-continue discipline, the postcondition of `wait` is the class invariant, `CI`. Thus, `CI` is the loop invariant.
- For `endRead`:
```algorithm
        {CI ∧ nr > 0}
        nr := nr – 1
        {CI}
        if nr = 0 then
            {CI} signal(writeOK) {CI}
        {CI}
```
> With the signal-and-continue discipline, the postcondition of `signal` is its precondition, `CI`. Thus, `CI` is the postcondition of the `if` statement.
- For `startWrite`:
```algorithm
        {CI}
        while nr > 0 or nw > 0 do
            wait(writeOK) {CI}
        {CI ∧ nr = 0 ∧ nw = 0}
        nw := nw + 1
        {CI}
```
> With the signal-and-continue discipline, the postcondition of `wait` is the class invariant, `CI`. Thus, `CI` is the loop invariant.
- For `endWrite`
```algorithm
        {CI ∧ nw > 0}
        nw := nw – 1
        {CI}
        signal(writeOK)
        {CI}
        signalAll(readOK)
        {CI}
```
> With the signal-and-continue discipline, the postcondition of `signal` is its precondition, `CI`. Thus, `CI` holds throughout the body.

When a writer ends, either a waiting writer or all waiting readers can continue. However, once a reader starts and there is a stream of overlapping readers, writers cannot start writing. Modify it to give preference to writers in this case. That is, if there are readers reading and a writer wants to write, no more readers are allowed to start reading. Also, after a writer finishes and both readers and writers are waiting, writers should go first.

YOUR ALGORITHM HERE

_Solution:_  
Here is a solution that uses a variable, `dw`, for the number of delayed writers:
```algorithm
monitor RWcontroller
    var nr, nw, dw = 0, 0, 0
    {(nr = 0 ∨ nw = 0) ∧ nw ≤ 1}
    var readOK: condition  {signalling condition:  nw = 0}
    var writeOK: condition  {signalling condition:  nr = 0 ∧ nw = 0}
    procedure startRead
        while nw > 0 or dw > 0 do wait(readOK)
        nr := nr + 1
    procedure endRead
        {nr > 0} nr := nr – 1
        if nr = 0 then signal(writeOK)
    procedure startWrite
        dw := dw + 1
        while nr > 0 or nw > 0 do wait(writeOK)
        nw, dw:= nw + 1, dw – 1
    procedure endWrite
        {nw > 0} nw := nw – 1
        if dw > 0 then signal(writeOK) else signalAll(readOK)
```

Add to the bodies of the modified procedures of  `RWcontroller` the complete annotation needed to show correctness. Argue for the correctness!

_Solution:_
- For `startRead`:
```algorithm
        {CI}
        while nw > 0 or dw > 0 do
            wait(readOK) 
        {CI ∧ nw = 0 ∧ dw = 0}
        nr := nr + 1
        {CI}
```
> With the signal-and-continue discipline, the postcondition of `wait` is the class invariant, `CI`. Thus, `CI` is the loop invariant.
- For `endRead`:
```algorithm
        {CI ∧ nr > 0}
        nr := nr – 1
        {CI}
        if nr = 0 then
            {CI} signal(writeOK) {CI}
        {CI}
```
> With the signal-and-continue discipline, the postcondition of `signal` is its precondition, `CI`. Thus, `CI` is the postcondition of the `if` statement.
- For `startWrite`:
```algorithm
        {CI}
        dw := dw + 1
        {CI ∧ dw > 0}
        while nr > 0 or nw > 0 do
            {dw > 0} wait(writeOK) {CI ∧ dw > 0}
        {CI ∧ nr = 0 ∧ nw = 0 ∧ dw > 0}
        nw, dw:= nw + 1, dw – 1
        {CI}
```
> With the signal-and-continue discipline, the postcondition of `wait` is the class invariant, `CI`. In addition, the precondition of `wait` is `dw > 0` and `dw` is modified by any other procedure. Thus, `CI ∧ dw > 0` is the loop invariant.
- For `endWrite`:
- For `endWrite`
```algorithm
        {CI ∧ nw > 0}
        nw := nw – 1
        {CI}
        if dw > 0 then
            {CI} signal(writeOK) {CI}
        else
            {CI} signalAll(readOK) {CI}
        {CI}
```
> With the signal-and-continue discipline, the postcondition of `signal` is its precondition, `CI`. Thus, `CI` holds throughout the body.